In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np

import nltk

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler

from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import GridSearchCV

#### Sistema de recomendación user-item:

- **def recomendacion_restaurante(user_id):**

> Ingresando el id de un usuario, deberíamos recibir una lista con 5 restaurantes recomendados para dicho usuario.

In [21]:
# Ruta del archivo
file_path = r'C:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\reviews.csv'

# Leer el archivo en un DataFrame
reviews = pd.read_csv(file_path)


In [22]:
# Eliminamos columnas irrelevantes
reviews = reviews.drop(['timestamp','sentiment_analysis'],axis=1)

In [ ]:
#filtramos por los negocios que son restaurantes o afines

In [26]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV

# Convertir el tipo de datos de las columnas user_id y business_id a string
reviews['user_id'] = reviews['user_id'].astype(str)
reviews['business_id'] = reviews['business_id'].astype(str)

# Especificamos el valor mínimo y el valor máximo de los ratings y el dataset a utilizar
reader = Reader(rating_scale=(reviews["rating"].min(), reviews["rating"].max()))

# El objeto Dataset de Surprise nos permite leer datos 
dataset = Dataset.load_from_df(reviews[['user_id', 'business_id', 'rating']], reader)

# Empleamos GridSearchCV con el algoritmo SVD y seteamos el parámetro refit a True con measures = ["rmse","fcp"]
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

gs = GridSearchCV(SVD, param_grid, measures=['fcp', "rmse"], cv=3, refit=True)

# Entrenamos el modelo
gs.fit(dataset)


In [27]:
# Mejor combinacion de parametros
gs.best_params


{'fcp': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6},
 'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

In [28]:
# Mejor Score
gs.best_score

{'fcp': 0.5337068346893777, 'rmse': 1.1767028884258932}

In [29]:
# Guardamos en una variable el modelo con mayor fcp
best_model = gs.best_estimator["fcp"]


In [30]:
reviews.head(2)

,user_id,rating,business_id
0,1.0147185615514871e+20,1,0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.1547723478903833e+20,5,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [33]:
# Prediccion para user_id : "1.0147185615514871e+20",  item_id : "0x8893863ea87bd5dd:0x9383ebf973e74abb

user_id = "1.1547723478903833e+20"
business_id = "0x8893863ea87bd5dd:0x9383ebf973e74abb"

pred = best_model.predict(user_id, business_id)

pred.est

4.074575159648792

In [37]:
from surprise.dump import dump

# Nombre del archivo para guardar el modelo
modelo_guardado = "modelo_user_item.pkl"

# Guardar el modelo entrenado
dump(modelo_guardado, algo=best_model)


In [39]:
from surprise.dump import load

def recomendacion_usuario(user_id, top_n=5):
    # Cargar el modelo entrenado desde el archivo guardado
    modelo_guardado = "modelo_user_item.pkl"
    loaded_model = load(modelo_guardado)[1]
    
    # Crear una lista vacía para almacenar las recomendaciones
    recomendaciones = []
    
    # Obtener todos los ítems que el usuario aún no ha calificado
    items_no_vistos = reviews[~reviews['business_id'].isin(reviews[reviews['user_id'] == user_id]['business_id'])]['business_id'].unique()
    
    # Realizar predicciones para cada ítem no visto
    for item_id in items_no_vistos:
        pred = loaded_model.predict(user_id, item_id)
        recomendaciones.append((item_id, pred.est))
    
    # Ordenar las recomendaciones en orden descendente según la calificación estimada
    recomendaciones.sort(key=lambda x: x[1], reverse=True)
    
    # Devolver los top N elementos recomendados
    return recomendaciones[:top_n]


In [41]:
recomendacion_usuario("1.1547723478903833e+20")

[('0x88e77f98a0d0d53f:0x43b542fc1f5dd4a0', 4.682763252289878),
 ('0x88d9a7c57c1ab3bd:0x6801bc5f3ac7787b', 4.678695264620291),
 ('0x88c2a1470c08b96d:0x85fa2dfbfa4b9d68', 4.663169516549163),
 ('0x88c2c32896691aff:0x8723493fe3c1e31a', 4.6592102702338964),
 ('0x88db391e3c770d6f:0xb0b7d298dc37cb14', 4.656393365826446)]

In [48]:
import pandas as pd
from surprise.dump import load

path = r"C:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\name_business.csv"

# Cargar el DataFrame con los nombres de los negocios
name_business = pd.read_csv(path)

# Función para recomendar restaurantes a un usuario
def recomendacion_usuario(user_id, top_n=5):
    # Cargar el modelo entrenado desde el archivo guardado
    modelo_guardado = "modelo_user_item.pkl"
    loaded_model = load(modelo_guardado)[1]
    
    # Crear una lista vacía para almacenar las recomendaciones
    recomendaciones = []
    
    # Obtener todos los ítems que el usuario aún no ha calificado
    items_no_vistos = reviews[~reviews['business_id'].isin(reviews[reviews['user_id'] == user_id]['business_id'])]['business_id'].unique()
    
    # Realizar predicciones para cada ítem no visto
    for item_id in items_no_vistos:
        pred = loaded_model.predict(user_id, item_id)
        recomendaciones.append((item_id, pred.est))
    
    # Ordenar las recomendaciones en orden descendente según la calificación estimada
    recomendaciones.sort(key=lambda x: x[1], reverse=True)
    
    # Mapear business_id con el nombre correspondiente
    recomendaciones_con_nombre = []
    for item_id, rating in recomendaciones[:top_n]:
        nombre = name_business[name_business['business_id'] == item_id]['name'].values[0]
        recomendaciones_con_nombre.append


In [49]:
recomendacion_usuario("1.1547723478903833e+20")


IndexError: index 0 is out of bounds for axis 0 with size 0